### create_data_to_analyze_study1_study2_v0.1.ipynb
- 最終更新: 2023/7/26, 8:22→2023/8/8, 13:22

- 研究1と研究2のデータ分析用のデータ作成

#### Google Driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import json

In [3]:
#classification_reportを辞書型で格納する関数の宣言
import re

first_row_pattern = "\s+"

def classification_report_to_dict(input_classification_report):
  """
  classification_reportを辞書型で格納する
  """
  classification_report_dict = dict()
  for idx, line in enumerate(input_classification_report.split("\n")):
    #評価値キーの準備
    if idx == 0:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      keys = line.split(" ")

      for tmp_key in keys:
        classification_report_dict[tmp_key] = dict()

    #評価値の取得
    if idx in [2, 3, 4]:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      keys = line.split(" ")

      #格納していく
      classification_report_dict["precision"][keys[0]] = keys[1]
      classification_report_dict["recall"][keys[0]] = keys[2]
      classification_report_dict["f1-score"][keys[0]] = keys[3]
      classification_report_dict["support"][keys[0]] = keys[4]

    elif idx == 6:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      keys = line.split(" ")

      #格納していく
      classification_report_dict["f1-score"][keys[0]] = keys[1]
      classification_report_dict["support"][keys[0]] = keys[2]

    elif idx in [7, 8]:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      line = re.sub("macro avg", "macro_avg", line)
      line = re.sub("weighted avg", "weighted_avg", line)
      keys = line.split(" ")

      #格納していく
      classification_report_dict["precision"][keys[0]] = keys[1]
      classification_report_dict["recall"][keys[0]] = keys[2]
      classification_report_dict["f1-score"][keys[0]] = keys[3]
      classification_report_dict["support"][keys[0]] = keys[4]

  return classification_report_dict

In [4]:
#テスト: 単発
classification_report_dir = "/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/study1/study1_answer_task_train_and_eval/Fold0/classification_report.txt"
with open(classification_report_dir, "r") as input_file:
  classification_report = input_file.read()

print(classification_report)
print(classification_report_to_dict(classification_report))

              precision    recall  f1-score   support

           2       0.34      0.69      0.46        16
           1       0.82      0.71      0.76       118
           0       0.79      0.79      0.79       110

    accuracy                           0.75       244
   macro avg       0.65      0.73      0.67       244
weighted avg       0.78      0.75      0.76       244

{'precision': {'2': '0.34', '1': '0.82', '0': '0.79', 'macro_avg': '0.65', 'weighted_avg': '0.78'}, 'recall': {'2': '0.69', '1': '0.71', '0': '0.79', 'macro_avg': '0.73', 'weighted_avg': '0.75'}, 'f1-score': {'2': '0.46', '1': '0.76', '0': '0.79', 'accuracy': '0.75', 'macro_avg': '0.67', 'weighted_avg': '0.76'}, 'support': {'2': '16', '1': '118', '0': '110', 'accuracy': '244', 'macro_avg': '244', 'weighted_avg': '244'}}


In [5]:
tmp_classification_df =  pd.DataFrame.from_dict(classification_report_to_dict(classification_report))
tmp_classification_df

,precision,recall,f1-score,support
2,0.34,0.69,0.46,16
1,0.82,0.71,0.76,118
0,0.79,0.79,0.79,110
macro_avg,0.65,0.73,0.67,244
weighted_avg,0.78,0.75,0.76,244
accuracy,NaN,NaN,0.75,244


In [6]:
tmp_classification_df_rev = tmp_classification_df.copy()
tmp_classification_df_rev["exp"] = "study1_answer_task_train_and_eval"
tmp_classification_df_rev

,precision,recall,f1-score,support,exp
2,0.34,0.69,0.46,16,study1_answer_task_train_and_eval
1,0.82,0.71,0.76,118,study1_answer_task_train_and_eval
0,0.79,0.79,0.79,110,study1_answer_task_train_and_eval
macro_avg,0.65,0.73,0.67,244,study1_answer_task_train_and_eval
weighted_avg,0.78,0.75,0.76,244,study1_answer_task_train_and_eval
accuracy,NaN,NaN,0.75,244,study1_answer_task_train_and_eval


In [7]:
#全体を処理して、研究1と研究2の結果をまとめる
#各Foldの平均をとる

#研究1
study_one_result_df = pd.DataFrame()

BASE_STUDY_ONE_RESULT_DIR = "/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/study1/"
EXP_STUDY_ONE_LIST = ["study1_answer_task_train_and_eval",
                      "study1_answer_rmtask_train_and_eval",
                      "study1_rubric_task_train_and_eval",
                      "study1_rubric_rmtask_train_and_eval",
                      "study1_answer_and_rubric_task_train_and_eval",
                      "study1_answer_and_rubric_rmtask_train_and_eval"]

for exp in EXP_STUDY_ONE_LIST:
  for num_fold in range(5):
    num_fold_key = "Fold" + str(num_fold)

    #classification_reportの読み込み
    exp_result_dir = BASE_STUDY_ONE_RESULT_DIR + exp + "/Fold" + str(num_fold) + "/classification_report.txt"
    with open(exp_result_dir, "r") as input_file:
      classification_report = input_file.read()
    classification_report_dict = classification_report_to_dict(classification_report)

    #データフレームにする
    classification_report_df = pd.DataFrame.from_dict(classification_report_dict)
    classification_report_df["exp"] = exp
    classification_report_df["Fold"] = num_fold_key

    study_one_result_df = pd.concat([study_one_result_df, classification_report_df])

study_one_result_df

,precision,recall,f1-score,support,exp,Fold
2,0.34,0.69,0.46,16,study1_answer_task_train_and_eval,Fold0
1,0.82,0.71,0.76,118,study1_answer_task_train_and_eval,Fold0
0,0.79,0.79,0.79,110,study1_answer_task_train_and_eval,Fold0
macro_avg,0.65,0.73,0.67,244,study1_answer_task_train_and_eval,Fold0
weighted_avg,0.78,0.75,0.76,244,study1_answer_task_train_and_eval,Fold0
...,...,...,...,...,...,...
1,0.83,0.83,0.83,118,study1_answer_and_rubric_rmtask_train_and_eval,Fold4
0,0.88,0.89,0.89,109,study1_answer_and_rubric_rmtask_train_and_eval,Fold4
macro_avg,0.73,0.72,0.72,243,study1_answer_and_rubric_rmtask_train_and_eval,Fold4
weighted_avg,0.83,0.83,0.83,243,study1_answer_and_rubric_rmtask_train_and_eval,Fold4


In [8]:
study_one_result_df.to_csv("/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/notebook/analysis/data_for_analysis/study_one_result_df.csv")

In [9]:
#研究2
study_two_result_df = pd.DataFrame()

BASE_STUDY_TWO_RESULT_DIR = "/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/study2/"
EXP_STUDY_TWO_LIST = ["study2_answer_train_and_eval", "study2_rubric_train_and_eval", "study2_answer_and_rubric_train_and_eval"]

for exp in EXP_STUDY_TWO_LIST:
  #classification_reportの読み込み
  exp_result_dir = BASE_STUDY_TWO_RESULT_DIR + exp + "/classification_report.txt"
  with open(exp_result_dir, "r") as input_file:
    classification_report = input_file.read()
  classification_report_dict = classification_report_to_dict(classification_report)

  #データフレームにする
  classification_report_df = pd.DataFrame.from_dict(classification_report_dict)
  classification_report_df["exp"] = exp

  study_two_result_df = pd.concat([study_two_result_df, classification_report_df])

study_two_result_df

,precision,recall,f1-score,support,exp
2,0.47,0.12,0.19,120,study2_answer_train_and_eval
1,0.78,0.85,0.82,630,study2_answer_train_and_eval
0,0.83,0.88,0.85,559,study2_answer_train_and_eval
macro_avg,0.69,0.62,0.62,1309,study2_answer_train_and_eval
weighted_avg,0.77,0.80,0.78,1309,study2_answer_train_and_eval
accuracy,NaN,NaN,0.80,1309,study2_answer_train_and_eval
2,0.16,0.38,0.23,120,study2_rubric_train_and_eval
1,0.64,0.59,0.61,630,study2_rubric_train_and_eval
0,0.74,0.60,0.66,559,study2_rubric_train_and_eval
macro_avg,0.51,0.52,0.50,1309,study2_rubric_train_and_eval


In [10]:
study_two_result_df.to_csv("/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/notebook/analysis/data_for_analysis/study_two_result_df.csv")